# Problem Statment
I would like to explore food options in the city of Toronoto.
Would like to perform clustering on all the restaurants in the city of Toronto and 
analyse most frequently visited places in a neighbourhood.
It would help to find out in which neighbourhooods different types of restaurants are located.
It would be easier for a new person in the city to know all the available food options by neighbourhood and decide to try out different cuisines.

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files



import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(url)
df = df[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
df['Borough'] = df['Borough'].apply(lambda x : np.nan if x == "Not assigned" else x)

In [4]:
df.dropna(axis=0,inplace=True)

In [5]:
df.reset_index(drop=True,inplace=True)

In [6]:
df.shape

(103, 3)

In [7]:

lat = pd.read_csv('Geospatial_Coordinates.csv')
lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df.sort_values(by='Postal Code',axis=0,inplace=True)

In [9]:
df = pd.concat([df,lat['Latitude'],lat['Longitude']],axis=1)
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476


In [10]:
address = 'Toronto City, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.65238435, -79.38356765.


In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [12]:
CLIENT_ID = '0XCVDGSRTZAVU2EFGXI20H2VWBOEQZ5DBSYBFA1CEF0M2E1S' # your Foursquare ID
CLIENT_SECRET = 'LLPM4BFMF2KZX5WR3HJE4G3NFT4CSHGVD0CCS2OJNVIM35DQ' # your Foursquare Secret
VERSION = '20210211' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
QUERY = "food"

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0XCVDGSRTZAVU2EFGXI20H2VWBOEQZ5DBSYBFA1CEF0M2E1S
CLIENT_SECRET:LLPM4BFMF2KZX5WR3HJE4G3NFT4CSHGVD0CCS2OJNVIM35DQ


In [13]:
df.loc[0, 'Neighbourhood']

'Parkwoods'

In [14]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.806686299999996, -79.19435340000001.


In [15]:


LIMIT = 10000 # limit of number of venues returned by Foursquare API

radius = 50000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT,QUERY)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=0XCVDGSRTZAVU2EFGXI20H2VWBOEQZ5DBSYBFA1CEF0M2E1S&client_secret=LLPM4BFMF2KZX5WR3HJE4G3NFT4CSHGVD0CCS2OJNVIM35DQ&v=20210211&ll=43.806686299999996,-79.19435340000001&radius=50000&limit=10000'

In [16]:

results = requests.get(url).json()

In [17]:
results

{'meta': {'code': 200, 'requestId': '60289f5a01bac51bb102e43d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 237,
  'suggestedBounds': {'ne': {'lat': 44.25668675000045,
    'lng': -78.57197147304028},
   'sw': {'lat': 43.35668584999954, 'lng': -79.81673532695974}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bca55e2511f9521b0a7afc7',
       'name': 'Port Union Waterfront Park',
       'location': {'address': 'Port Union Road, South End',
        'crossStreet': 'Lake Ontario',
        'lat': 43.774967388607294,
        'lng': -79.13439273834229,
        'labeledLatLngs': [{

In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Port Union Waterfront Park,Park,43.774967,-79.134393
1,Mona's Roti,Caribbean Restaurant,43.791613,-79.251015
2,Frenchman's Bay,Beach,43.813402,-79.081819
3,Chris Jerk Caribbean Bistro,Caribbean Restaurant,43.789660,-79.302340
4,Scarborough Bluffs,Beach,43.707780,-79.237239


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,QUERY)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(1652, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Parkwoods,43.806686,-79.194353,Meena's Fine Foods,43.804476,-79.199753,Indian Restaurant
2,"Regent Park, Harbourfront",43.763573,-79.188711,Doug's Bakery,43.762400,-79.192285,Bakery
3,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
4,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [73]:
toronto_venues.groupby('Venue Category').count()['Neighborhood'].to_frame().sort_values(by='Neighborhood',ascending=False).head(10)

,Neighborhood
Venue Category,
Café,139
Restaurant,126
Pizza Place,103
Sandwich Place,92
Italian Restaurant,80
Sushi Restaurant,70
Bakery,68
Japanese Restaurant,61
Fast Food Restaurant,53


In [87]:
toronto_venues.groupby('Venue Category').count()['Neighborhood'].to_frame().sort_values(by='Neighborhood').head(10)

,Neighborhood
Venue Category,
Afghan Restaurant,1
Food,1
German Restaurant,1
Doner Restaurant,1
Hakka Restaurant,1
Korean BBQ Restaurant,1
Mac & Cheese Joint,1
Malay Restaurant,1
North Indian Restaurant,1


In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,16,16,16,16,16,16
"Alderwood, Long Branch",1,1,1,1,1,1
"Bathurst Manor, Wilson Heights, Downsview North",9,9,9,9,9,9
Bayview Village,10,10,10,10,10,10
"Bedford Park, Lawrence Manor East",72,72,72,72,72,72
Berczy Park,1,1,1,1,1,1
"Birch Cliff, Cliffside West",96,96,96,96,96,96
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",2,2,2,2,2,2


In [85]:
toronto_venues.groupby('Neighborhood').count()['Venue'].to_frame().sort_values(by='Venue',ascending=False).head(10)

,Venue
Neighborhood,
Downsview,132
Westmount,100
Lawrence Park,100
"High Park, The Junction South",98
Studio District,98
"Birch Cliff, Cliffside West",96
"Willowdale, Willowdale East",83
"Bedford Park, Lawrence Manor East",72
"Humberlea, Emery",66


In [86]:
toronto_venues.groupby('Neighborhood').count()['Venue'].to_frame().sort_values(by='Venue').head(10)

,Venue
Neighborhood,
"Alderwood, Long Branch",1
"Runnymede, The Junction North",1
"The Annex, North Midtown, Yorkville",1
"Richmond, Adelaide, King",1
Berczy Park,1
"Parkview Hill, Woodbine Gardens",1
"Dufferin, Dovercourt Village",1
"Islington Avenue, Humber Valley Village",1
"Golden Mile, Clairlea, Oakridge",1


In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 94 uniques categories.


In [28]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Creperie,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Hot Dog Joint,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Korean BBQ Restaurant,Korean Restaurant,Latin American Restaurant,Mac & Cheese Joint,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Peking Duck Restaurant,Persian Restaurant,Pizza Place,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shawarma Place,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
toronto_onehot.shape

(1652, 95)

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Creperie,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Hot Dog Joint,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Korean BBQ Restaurant,Korean Restaurant,Latin American Restaurant,Mac & Cheese Joint,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Peking Duck Restaurant,Persian Restaurant,Pizza Place,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shawarma Place,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Agincourt,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.125000,0.000000,0.062500,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.062500,0.062500,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000
1,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.00000,0.111111,0.0,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000

In [31]:

toronto_grouped.shape

(88, 95)

In [32]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0                 Café  0.25
1       Sandwich Place  0.12
2       Breakfast Spot  0.12
3               Bakery  0.12
4  Japanese Restaurant  0.06


----Alderwood, Long Branch----
                             venue  freq
0                      Pizza Place   1.0
1                Afghan Restaurant   0.0
2  Molecular Gastronomy Restaurant   0.0
3                Polish Restaurant   0.0
4                       Poke Place   0.0


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0      Restaurant  0.11
1           Diner  0.11
2   Deli / Bodega  0.11
3  Sandwich Place  0.11
4     Pizza Place  0.11


----Bayview Village----
                  venue  freq
0     Indian Restaurant   0.2
1        Sandwich Place   0.2
2  Fast Food Restaurant   0.2
3          Burger Joint   0.1
4            Restaurant   0.1


----Bedford Park, Lawrence Manor East----
                venue  freq
0          Restaurant  0.11
1                Café

4                       Poke Place   0.0


----Kennedy Park, Ionview, East Birchmount Park----
              venue  freq
0  Sushi Restaurant  0.19
1    Sandwich Place  0.14
2       Pizza Place  0.14
3      Burger Joint  0.10
4    Breakfast Spot  0.10


----Kensington Market, Chinatown, Grange Park----
                venue  freq
0         Pizza Place  0.16
1                Café  0.12
2    Sushi Restaurant  0.12
3               Diner  0.08
4  Italian Restaurant  0.08


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                venue  freq
0          Restaurant  0.07
1  Mexican Restaurant  0.05
2              Bakery  0.05
3         Pizza Place  0.05
4               Diner  0.05


----Lawrence Manor, Lawrence Heights----
                   venue  freq
0      Indian Restaurant  0.33
1  Korean BBQ Restaurant  0.33
2     Mexican Restaurant  0.33
3    Moroccan Restaurant  0.00
4      Polish Restaurant  0.00


----Lawrence Park----
                 venue  fr

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Café,Bakery,Sandwich Place,Breakfast Spot,Burrito Place,Restaurant,Pizza Place,Vietnamese Restaurant,Italian Restaurant,Japanese Restaurant
1,"Alderwood, Long Branch",Pizza Place,Wings Joint,Filipino Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Deli / Bodega,Diner,Fried Chicken Joint,Chinese Restaurant,Restaurant,Sandwich Place,Sushi Restaurant,Pizza Place,Middle Eastern Restaurant,Ethiopian Restaurant
3,Bayview Village,Indian Restaurant,Fast Food Restaurant,Sandwich Place,Restaurant,Middle Eastern Restaurant,Burger Joint,Pizza Place,Food Truck,Dim Sum Restaurant,Diner
4,"Bedford Park, Lawrence Manor East",Restaurant,Café,Italian Restaurant,Sushi Restaurant,Diner,Breakfast Spot,American Restaurant,Gastropub,Bakery,Sandwich Place


In [106]:
neighborhoods_venues_sorted.loc[(neighborhoods_venues_sorted['Neighborhood'] == 'Downsview') | (neighborhoods_venues_sorted['Neighborhood'] == 'Westmount') | (neighborhoods_venues_sorted['Neighborhood'] == 'Lawrence Park')| (neighborhoods_venues_sorted['Neighborhood'] == 'High Park, The Junction South')]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,2,Downsview,Restaurant,Café,Italian Restaurant,Bakery,Sandwich Place,Salad Place,Deli / Bodega,Seafood Restaurant,Japanese Restaurant,Asian Restaurant
33,2,"High Park, The Junction South",Sandwich Place,Italian Restaurant,Restaurant,Bakery,Sushi Restaurant,Café,Seafood Restaurant,Pizza Place,Breakfast Spot,Japanese Restaurant
43,2,Lawrence Park,Restaurant,Café,Italian Restaurant,Salad Place,American Restaurant,Bakery,Breakfast Spot,Sandwich Place,Japanese Restaurant,Seafood Restaurant
79,2,Westmount,Café,Restaurant,American Restaurant,Italian Restaurant,Salad Place,Bakery,Deli / Bodega,Sandwich Place,Seafood Restaurant,Japanese Restaurant


In [35]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 3, 2, 2, 2, 2, 2, 2, 0, 1])

In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.806686,-79.194353,1.0,Indian Restaurant,Fast Food Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
1,M4A,North York,Victoria Village,43.784535,-79.160497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,2.0,Donut Shop,Restaurant,Bakery,Shawarma Place,Breakfast Spot,Pizza Place,Mexican Restaurant,Wings Joint,Fast Food Restaurant,Diner
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917,2.0,Indian Restaurant,Korean BBQ Restaurant,Mexican Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476,2.0,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Caribbean Restaurant,Bakery,Filipino Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [51]:
toronto_merged.dropna(inplace=True)

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)



In [53]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [(colors.rgb2hex(i)) for i in colors_array]



In [54]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    

In [55]:
folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color = rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,East York,0.0,Vietnamese Restaurant,Pizza Place,Caribbean Restaurant,Falafel Restaurant,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
34,North York,0.0,Portuguese Restaurant,Pizza Place,French Restaurant,Fast Food Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
35,East York,0.0,Pizza Place,Breakfast Spot,Gastropub,Café,Filipino Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
37,West Toronto,0.0,Burger Joint,Pizza Place,Asian Restaurant,Wings Joint,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
45,North York,0.0,Breakfast Spot,Asian Restaurant,Sandwich Place,Pizza Place,German Restaurant,Fast Food Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
72,North York,0.0,Pizza Place,Japanese Restaurant,Asian Restaurant,Bakery,Sushi Restaurant,Filipino Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
76,Mississauga,0.0,Pizza Place,Bakery,Café,Middle Eastern Restaurant,Brazilian Restaurant,Portuguese Restaurant,French Restaurant,Gluten-free Restaurant,Diner,Doner Restaurant
81,West Toronto,0.0,Breakfast Spot,Pizza Place,Caribbean Restaurant,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
89,Etobicoke,0.0,Pizza Place,Burger Joint,Sandwich Place,Wings Joint,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
95,Scarborough,0.0,Pizza Place,Fried Chicken Joint,Café,Wings Joint,Fast Food Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


In [57]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Indian Restaurant,Fast Food Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
17,Etobicoke,1.0,Mediterranean Restaurant,Fast Food Restaurant,Wings Joint,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
25,Downtown Toronto,1.0,BBQ Joint,Fast Food Restaurant,Wings Joint,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
63,York,1.0,Fast Food Restaurant,Wings Joint,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
80,Downtown Toronto,1.0,Fast Food Restaurant,Restaurant,Fried Chicken Joint,Sandwich Place,Wings Joint,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
87,Downtown Toronto,1.0,Fast Food Restaurant,Pizza Place,Restaurant,Burrito Place,Wings Joint,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2.0,Donut Shop,Restaurant,Bakery,Shawarma Place,Breakfast Spot,Pizza Place,Mexican Restaurant,Wings Joint,Fast Food Restaurant,Diner
3,North York,2.0,Indian Restaurant,Korean BBQ Restaurant,Mexican Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
4,Downtown Toronto,2.0,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Caribbean Restaurant,Bakery,Filipino Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
6,Scarborough,2.0,Chinese Restaurant,Bakery,Sri Lankan Restaurant,Wings Joint,Filipino Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
7,North York,2.0,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Bakery,Fried Chicken Joint,Noodle House,Indian Restaurant,Thai Restaurant,Italian Restaurant,Gastropub
8,East York,2.0,American Restaurant,Wings Joint,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
9,Downtown Toronto,2.0,Pizza Place,Café,Mediterranean Restaurant,Fast Food Restaurant,Wings Joint,Filipino Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
10,North York,2.0,Indian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Greek Restaurant,Fast Food Restaurant,Diner,Doner Restaurant,Donut Shop,Hakka Restaurant,Dumpling Restaurant
11,Etobicoke,2.0,Middle Eastern Restaurant,Vietnamese Restaurant,Sandwich Place,Bakery,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
12,Scarborough,2.0,Breakfast Spot,Chinese Restaurant,Latin American Restaurant,Fish & Chips Shop,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant


In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,3.0,Pizza Place,Wings Joint,Filipino Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
24,Downtown Toronto,3.0,Pizza Place,Wings Joint,Filipino Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
93,Etobicoke,3.0,Pizza Place,Wings Joint,Filipino Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant


In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,West Toronto,4.0,Asian Restaurant,Wings Joint,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
